# 📘 Analyse des connexions optogénétiques via LFP (version améliorée)
Ce notebook est une version enrichie du pipeline initial. Il permet :
- d'extraire automatiquement les paramètres des réponses évoquées (amplitude, latence, aire),
- de visualiser les évolutions par région et par âge,
- de générer des tableaux de résultats prêts à être utilisés pour l’analyse statistique.

In [ ]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os
from IPython.display import display
from ipyfilechooser import FileChooser
from scipy.stats import zscore
import json
import matplotlib.cm as cm
import IPython
import ast

%matplotlib widget

In [ ]:
#Load LFP coordinates 
notebook_path = Path("/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]))
Channels = f'{notebook_path.parent}/_LFP_coordinates_of_all_mice.csv'
all_LFPcoordinates = pd.read_csv(Channels, index_col=0)

In [ ]:
try: # tries to retrieve dpath either from a previous run or from a previous notebook
    %store -r dpath
except:
    print("the path was not defined in store")
    dpath = "//10.69.168.1/crnldata/forgetting/"

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Go inside the folder containing the LFP raw file</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

In [ ]:
folder_base = Path(dpath) 
miceIDflipped=[]

if Path(f'{folder_base}\DataFrame_rawdataDS.pkl').exists(): # prefer loading downsample file over original file
    print('DataFrame_rawdataDS.pkl file')
    LFPfile = Path(f'{folder_base}\DataFrame_rawdataDS.pkl')
    LFPs_df = pd.read_pickle(LFPfile)
    samplerate = 1000 
    numchannel = LFPs_df.shape[1]
    rec_ch_list = LFPs_df.columns.values
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStampsDS.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS)  
elif Path(f'{folder_base}\continuous.dat').exists():
    print('continuous.dat file')
    LFPfile = Path(f'{folder_base}\continuous.dat')
    DataRec = np.fromfile(LFPfile, dtype="int16")
    filepath = Path(os.path.join(folder_base.parent.parent, f'structure.oebin'))
    with open(filepath) as f:
        metadata = json.load(f)
    samplerate = metadata['continuous'][0]['sample_rate']  
    numchannel = metadata['continuous'][0]['num_channels'] 
    rec_ch_list = np.array([int(''.join(c for c in metadata['continuous'][0]['channels'][x]['channel_name'] if c.isdigit()))-1 for x in range(0, len(metadata['continuous'][0]['channels']))])
    DataRec = DataRec.reshape(-1,numchannel)
    print('Metadata found')
    # Load LFPs timestamps 
    for file_pathTS in folder_base.parent.parent.glob('**/continuous/*/timeStamps.npy'):
        print('LFPs timestamps file found')
        LFPtimestamps = np.load(file_pathTS) 
    LFPs_df=pd.DataFrame(DataRec, columns=rec_ch_list) 
else: 
    print('no LFPs file found')

print('sample rate =', samplerate, 'Hz')
print(numchannel, 'channels recorded')
print(round(LFPs_df.shape[0]/samplerate/60), 'min of recording')

In [ ]:
nb_decimal=4 # 4 = 0.1ms precision / 3 = 1ms precision

# Load TTLs
TTL_Opto_duration=[]
for file_pathTTL in folder_base.parent.parent.glob('**/TTL/timeStamps.npy'):
    print('TTL opto file = ', file_pathTTL)
    TTL_Opto_o = np.load(file_pathTTL)
    TTL_Opto_duration =[round(TTL_Opto_o[i+1] - TTL_Opto_o[i],nb_decimal) for i in range(len(TTL_Opto_o) - 1)[::2]]
    TTL_Opto= TTL_Opto_o[::2] # remove the TTL for laser OFF, only keep TTL for laser ON. CAUTION /!/ works only if it started with a TTL for laser ON
    print(TTL_Opto.shape[0], 'opto stimulations')

In [ ]:
if samplerate > 1000:
    new_sampling_rate = 1000 # Hz
    Nmber_points = int(np.shape(LFPs_df)[0] * new_sampling_rate / samplerate)
    LFPs_df_DS = pd.DataFrame(signal.resample(LFPs_df, Nmber_points, axis = 0), columns=LFPs_df.columns.values)
    LFPtimestampsDS = LFPtimestamps[::int(samplerate/new_sampling_rate)][:-1]
    samplerate = new_sampling_rate
    LFPs_df_DS.to_pickle(f'{LFPfile.parent}/DataFrame_rawdataDS.pkl')
    np.save(f'{file_pathTS.parent}/timeStampsDS.npy', LFPtimestampsDS)
    LFPs_df = LFPs_df_DS
    LFPtimestamps = LFPtimestampsDS
# eventually delete original files to gain space

In [ ]:
mouse = []
pos_mice = []
for mouse_name in all_LFPcoordinates.index:
    if mouse_name in LFPfile.__str__():
        mouse.append(mouse_name)
        pos_mice.append(LFPfile.__str__().find(mouse_name)) 
mouse = [x for _, x in sorted(zip(pos_mice, mouse))] # sort mouse in the same order as they appear in the path

if len(mouse) > 1: # found multiple mouse name in the path
    if max(rec_ch_list) <= 31: # no channels superior to 32, so only one mouse recorded
        id = 0 # change to 0 to see the first mouse name found, 1 the second, etc
        ID = 0
        print(f"/!\ Mutliple mice name found in the path but only mouse recorded = {mouse}. The n°{id+1} was choosen automatically = {mouse[id]}.")
        mouse = mouse[id]
    else:
        ###################################################################################################################################
        ID = 2 # choose 0 to see the first mouse recorded, 1 the second, 2 the third, 3 the fourth (only 4 mice can be recorded at the same time)
        ###################################################################################################################################
        print(f"/!\ Mutliple mice recorded at the same time = {mouse}. The n°{ID+1} was choosen automatically = {mouse[ID]}.")
        mouse = mouse[ID] 
elif len(mouse) == 1: # found only one mouse name in the path
    ID = 0
    mouse = mouse[ID]
    
all_LFPcoordinates= all_LFPcoordinates.astype(str)
for region in all_LFPcoordinates.loc[mouse].index:
    locals()[f'{region}_0']=[]
    locals()[f'{region}_1']=[]
    locals()[f'{region}_0ch']=[]
    locals()[f'{region}_1ch']=[]

RecordedArea = []
ChoosenChannels = []
combined = []
if mouse:
    rec_ch_list_mouse = [value for value in rec_ch_list if 0+(ID*32) <= value <= 31+(ID*32)]
    for rec_ch in rec_ch_list_mouse:
        for idx, LFPcoord_str in enumerate(all_LFPcoordinates.loc[mouse]):
            region = all_LFPcoordinates.loc[mouse].index[idx]
            if LFPcoord_str != 'nan' and region != 'EMG':
                LFPcoord = LFPcoord_str.split('_')[:2] # only take into account the 2 first of electrode of that region 
                num_ch = np.where(str(rec_ch-(ID*32)) == np.array(LFPcoord))[0]
                if len(num_ch)>0:
                    region=all_LFPcoordinates.loc[mouse].index[idx]
                    LFP=locals()[f'{region}_0']
                    if len(LFP)>0:
                        LFP= np.array(LFPs_df[(rec_ch)])
                        locals()[f'{region}_1']=LFP
                        locals()[f'{region}_1ch']=rec_ch
                    else:
                        LFP= np.array(LFPs_df[(rec_ch)])
                        locals()[f'{region}_0']=LFP
                        locals()[f'{region}_0ch']=rec_ch
                    break
                continue
    
    for region in all_LFPcoordinates.loc[mouse].index:
        for n in range(0,2,1):
            LFP=locals()[f'{region}_{n}']
            LFP_ch=locals()[f'{region}_{n}ch']
            if len(LFP)>0:
                combined=zscore(LFP[:,np.newaxis]) if len(combined)==0 else np.append(combined, zscore(LFP[:,np.newaxis]), axis=1)
                RecordedArea.append(f'{region}_{n}') 
                ChoosenChannels.append(LFP_ch) 
else:
    print("/!\ No mouse name found in the path OR in the csv file '_LFP_coordinates_of_all_mice.csv'")
    mouse = '' # fill mouse name
    RecordedArea = ['PFC','S1'] 
    PFC_0 = LFPs_df[0]
    S1_1 = LFPs_df[1]
    combined = np.stack([zscore(PFC_0), zscore(S1_1)], axis=1)

print(mouse)
print(RecordedArea)
print(ChoosenChannels) 

In [ ]:
Selected_region='PFC_1' # to change
SelectedLFP=locals()[Selected_region]

In [ ]:

# 📐 Extraction automatique des paramètres des réponses évoquées
from scipy.integrate import simps

results = []
window = (0, 50)  # en ms pour fenêtre de mesure post-TTL

for i, ttl in enumerate(TTL_Opto):
    epoch = AllEPSPs[i]  # (time x region)
    for region_idx, region_name in enumerate(RecordedArea):
        trace = epoch[:, region_idx]
        time_axis = np.linspace(-500, 500, len(trace))  # en ms

        # Zoom sur fenêtre d'intérêt (0–50 ms après stimulation)
        mask = (time_axis >= window[0]) & (time_axis <= window[1])
        trace_window = trace[mask]
        time_window = time_axis[mask]

        # Paramètres
        min_amp = np.min(trace_window)
        min_lat = time_window[np.argmin(trace_window)]
        area = simps(trace_window, time_window)  # aire sous la courbe

        results.append({
            'TTL_index': i,
            'Region': region_name,
            'Amplitude': min_amp,
            'Latence_ms': min_lat,
            'Aire': area,
            'Mouse': mouse,
            'Filename': folder_base.parts[-6],
        })

df_results = pd.DataFrame(results)
df_results.head()


In [ ]:

# 📊 Visualisation des paramètres extraits
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 4))
sns.boxplot(data=df_results, x="Region", y="Amplitude")
plt.title("Amplitude des réponses évoquées par région")
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
sns.boxplot(data=df_results, x="Region", y="Latence_ms")
plt.title("Latence des réponses évoquées par région")
plt.grid()
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
sns.boxplot(data=df_results, x="Region", y="Aire")
plt.title("Aire sous la courbe des réponses évoquées")
plt.grid()
plt.tight_layout()
plt.show()
